In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [2]:
from functools import partial

In [59]:
def random_partition(n, k, rand_generator):
    ''' Generate k values that sum to n '''
    p = []
    for i in range(k):
        v = rand_generator()
        p.append(min(n, v) if i < k-1 else n)
        n -= p[-1]
    return p
        
def generate_n_nodes(nodes, levels):    
#     n_nodes = [nodes // levels] * (levels - 1) + \
#               [nodes % levels + nodes // levels]
    offset = int(0.25 * (nodes // levels) + 1)
    n_nodes = random_partition(nodes, levels, 
                    partial(np.random.randint, low=max(1,nodes // levels - offset),
                                              high=nodes // levels + offset))
    assert sum(n_nodes) == nodes # partition
    return n_nodes
    
def generate_supply_demand(total_items, nodes):
    res = []
    for total_item in total_items: 
        res.append(random_partition(
                total_item, nodes, 
                partial(np.random.poisson, lam=total_item // nodes)))
        assert sum(res[-1]) == total_item # partition
    return res
    
def generate_dataset(nodes, levels, total_supplies, total_demands, 
                     transp_costs, random_state=None):
    if random_state:
        np.random.seed(random_state)
        
    # Generate number of nodes per level (n_levels)
    n_nodes = generate_n_nodes(nodes, levels)
    
    # Generate supplies for each item (n_items, n_nodes)
    supplies = generate_supply_demand(total_supplies, n_nodes[0])
    
    # Generate demands for each item (n_items, n_nodes)
    demands = generate_supply_demand(total_demands, n_nodes[-1])
    
    # Generate costs (n_levels-1, n_nodes[i], n_nodes[i+1])
    costs = [np.random.randint(*transp_costs, (n_nodes[i], n_nodes[i+1])) \
                for i in range(levels-1)]
    
    # Generate capacities (n_levels, n_nodes[i])
    max_flow = max(sum(total_supplies), sum(total_demands))
    min_flow = min(sum(total_supplies), sum(total_demands))
    capacities = [list(np.sum(supplies, axis=0))] + \
                 [random_partition(max_flow + np.random.poisson(int(0.1 * max_flow)), n, 
                     partial(np.random.poisson, lam=max_flow // n)) \
                     for n in n_nodes[1:-1]] + \
                 [list(np.sum(demands, axis=0))]
    for cap in capacities:
        assert min_flow <= sum(cap)
    
    dummy_supplies = [max(0, d - s) for s,d in zip(total_supplies, total_demands)]
    dummy_demands = [max(0, s - d) for s,d in zip(total_supplies, total_demands)]
    
    # Supply-demand + dummies (supply-demand) (n_items, n_nodes+1)
    supplies = [sup + [dum] for sup,dum in zip(supplies, dummy_supplies)]
    demands = [dem + [dum] for dem,dum in zip(demands, dummy_demands)]

    return n_nodes, supplies, demands, costs, capacities

In [60]:
n_nodes, supplies, demands, costs, capacities = generate_dataset(
                nodes=10, levels=4, total_supplies=[25,30], 
                total_demands=[28,25], transp_costs=(10,40), random_state=42)

In [61]:
n_nodes

[1, 2, 1, 6]

In [62]:
supplies

[[25, 3], [30, 0]]

In [63]:
demands

[[4, 4, 4, 2, 5, 9, 0], [5, 3, 5, 7, 1, 4, 5]]

In [64]:
capacities

[[55], [32, 26], [57], [9, 7, 9, 9, 6, 13]]

In [65]:
[cost.shape for cost in costs]

[(1, 2), (2, 1), (1, 6)]

In [66]:
n_nodes

[1, 2, 1, 6]

In [67]:
' '.join([f'T{i+1}{j+1}' for i in range(len(n_nodes[1:-1])) \
                        for j in range(n_nodes[1:-1][i])])

'T11 T12 T21'

In [68]:
supply_trans = ' '.join([f'(S{j+1},T1{k+1})' \
                         for j in range(n_nodes[0]) \
                         for k in range(n_nodes[1])])
trans_trans = ' '.join([f'(T{i+1}{j+1},T{i+2}{k+1})' \
                         for i in range(len(n_nodes[1:-1])-1) \
                         for j in range(n_nodes[1:-1][i]) \
                         for k in range(n_nodes[1:-1][i+1])])
trans_demand = ' '.join([f'(T{len(n_nodes[1:-1])}{j+1},D{k+1})' \
                         for j in range(n_nodes[-2]) \
                         for k in range(n_nodes[-1])])

In [69]:
supply_trans

'(S1,T11) (S1,T12)'

In [70]:
trans_demand

'(T21,D1) (T21,D2) (T21,D3) (T21,D4) (T21,D5) (T21,D6)'

In [71]:
trans_trans

'(T11,T21) (T12,T21)'

In [80]:
def print_items(n_items, fout):
    items = ' '.join([f'I{i+1}' for i in range(n_items)])
    print(f'set I := {items};', file=fout)        
    
def print_nodes(n_nodes, fout):
    supply = ' '.join([f'S{i+1}' for i in range(n_nodes[0])])
    trans = ' '.join([f'T{i+1}{j+1}' for i in range(len(n_nodes[1:-1])) \
                                     for j in range(n_nodes[1:-1][i])])
    demand = ' '.join([f'D{i+1}' for i in range(n_nodes[-1])])
    print(f'set ST := {supply} {trans};', file=fout)
    print(f'set D := {demand};', file=fout)
    print(f'set DU := dummy_sup dummy_dem;\n', file=fout)

def print_edges(set_name, n_nodes, fout, dummies=False):
    print(f'set {set_name} := ', file=fout)
    supply_trans = ' '.join([f'(S{j+1},T1{k+1})' \
                             for j in range(n_nodes[0]) \
                             for k in range(n_nodes[1])])
    trans_trans = ' '.join([f'(T{i+1}{j+1},T{i+2}{k+1})' \
                             for i in range(len(n_nodes[1:-1])-1) \
                             for j in range(n_nodes[1:-1][i]) \
                             for k in range(n_nodes[1:-1][i+1])])
    trans_demand = ' '.join([f'(T{len(n_nodes[1:-1])}{j+1},D{k+1})' \
                             for j in range(n_nodes[-2]) \
                             for k in range(n_nodes[-1])])
    print(f'   {supply_trans}', file=fout)
    print(f'   {trans_trans}', file=fout)
    print(f'   {trans_demand}', file=fout)
    
    if dummies:
        dummy_supply = ' '.join(f'(S{j+1},dummy_dem)' \
                                 for j in range(n_nodes[0]))
        dummy_demand = ' '.join(f'(dummy_sup,D{j+1})' \
                                 for j in range(n_nodes[-1]))
        print(f'   {dummy_supply}', file=fout)
        print(f'   {dummy_demand}', file=fout)
        
    print(';\n', file=fout)
    
def print_transp_cost(n_nodes, costs, fout):
    supply_trans = ' '.join([f'S{j+1} T1{k+1} {costs[0][j][k]}' \
                             for j in range(n_nodes[0]) \
                             for k in range(n_nodes[1])])
    trans_trans = ' '.join([f'T{i+1}{j+1} T{i+2}{k+1} {costs[i+1][j][k]}' \
                             for i in range(len(n_nodes[1:-1])-1) \
                             for j in range(n_nodes[1:-1][i]) \
                             for k in range(n_nodes[1:-1][i+1])])
    trans_demand = ' '.join([f'T{len(n_nodes[1:-1])}{j+1} D{k+1} {costs[-1][j][k]}' \
                             for j in range(n_nodes[-2]) \
                             for k in range(n_nodes[-1])])    
    print(f'param transp_cost := {supply_trans} {trans_trans} {trans_demand};\n', 
                             file=fout)
    
def print_supply_demand(n_nodes, supplies, demands, fout):
    supply = ' '.join([f'S{i+1}' for i in range(n_nodes[0])])
    trans = ' '.join([f'T{i+1}{j+1}' for i in range(len(n_nodes[1:-1])) \
                                     for j in range(n_nodes[1:-1][i])])
    demand = ' '.join([f'D{i+1}' for i in range(n_nodes[-1])])
    print(f'param supply_demand (tr): {supply} {trans} {demand} '
           'dummy_sup dummy_dem := ', file=fout)
    for i in range(len(supplies)):
        supply_demand = ' '.join([str(x) for x in supplies[i][:-1] + \
                                 [0]*sum(n_nodes[1:-1]) + \
                                 [-x for x in demands[i][:-1]] + \
                                 [supplies[i][-1]] + [-demands[i][-1]]])
        print(f'    I{i+1} {supply_demand}', file=fout)
    print(';\n', file=fout)
    
def print_capacities(n_nodes, capacities, fout):
    supply = ' '.join([f'S{i+1} {capacities[0][i]}' \
                              for i in range(n_nodes[0])])
    trans = ' '.join([f'T{i+1}{j+1} {capacities[i+1][j]}' 
                              for i in range(len(n_nodes[1:-1])) \
                              for j in range(n_nodes[1:-1][i])])
    demand = ' '.join([f'D{i+1} {capacities[-1][i]}' 
                              for i in range(n_nodes[-1])])
    print(f'param node_capacity := {supply} {trans} {demand};\n', 
                              file=fout)
    
def generate_ampl(n_nodes, supplies, demands, costs, capacities, 
                  output_file='data/model_data.mod'):
    with open('data/model.mod', 'r') as f_model, \
         open(output_file, 'w') as fout:
        print(f_model.read(), file=fout)
        print('data;', file=fout)
        print_items(len(supplies), fout)
        print_nodes(n_nodes, fout)        
        print_edges('E', n_nodes, fout)
        print_edges('EDU', n_nodes, fout, dummies=True)
        print_transp_cost(n_nodes, costs, fout)
        print_supply_demand(n_nodes, supplies, demands, fout)
        print_capacities(n_nodes, capacities, fout)
        

In [81]:
supplies

[[25, 3], [30, 0]]

In [82]:
capacities

[[55], [32, 26], [57], [9, 7, 9, 9, 6, 13]]

In [83]:
demands

[[4, 4, 4, 2, 5, 9, 0], [5, 3, 5, 7, 1, 4, 5]]

In [84]:
generate_ampl(n_nodes, supplies, demands, costs, capacities)

In [85]:
generate_ampl(n_nodes, supplies, demands, costs, capacities,
              output_file='modelos/test.mod')

## Test Library

In [96]:
n_nodes, supplies, demands, costs, capacities = generate_dataset(
                nodes=40, levels=5, total_supplies=[500,400,300], 
                total_demands=[400,600,400], transp_costs=(10,100), 
                random_state=42)

In [97]:
generate_ampl(n_nodes, supplies, demands, costs, capacities,
              output_file='modelos/test.mod')

In [91]:
n_nodes

[6, 7, 4, 6, 7]